In [1]:
import json
from pymongo import MongoClient

import uuid
from pprint import pprint

In [2]:
# generamos los datos localmente b

clientes = [
    {
        'nombre': 'Pedro',
        'apellidos': 'Cuenca',
        'ID': str(uuid.uuid4().int)[:7],
        'dinero_gastado': 0.0,
        'num_pedidos': 0,
        'pedidos': []
    },
    {
        'nombre': 'Ana',
        'apellidos': 'Martínez',
        'ID': str(uuid.uuid4().int)[:7],
        'dinero_gastado': 0.0,
        'num_pedidos': 0,
        'pedidos': []
    },
    {
        'nombre': 'Alicia',
        'apellidos': 'Romero',
        'ID': str(uuid.uuid4().int)[:7],
        'dinero_gastado': 0.0,
        'num_pedidos': 0,
        'pedidos': []
    },
    # Agrega más clientes si es necesario
]

# Creamos otra lista de  diccionarios con los pedidos

pedidos = [
    {
        'codigo_pedido': str(uuid.uuid4().int)[:7],
        'fecha': '2023-05-18',
        'codigo_cliente': clientes[0]['ID'],
        'importe_total': 100.0
    },
    {
        'codigo_pedido': str(uuid.uuid4().int)[:7],
        'fecha': '2023-05-17',
        'codigo_cliente': clientes[0]['ID'],
        'importe_total': 150.0
    },
    {
        'codigo_pedido': str(uuid.uuid4().int)[:7],
        'fecha': '2023-05-16',
        'codigo_cliente': clientes[1]['ID'],
        'importe_total': 200.0
    },
    {
        'codigo_pedido': str(uuid.uuid4().int)[:7],
        'fecha': '2023-05-16',
        'codigo_cliente': clientes[1]['ID'],
        'importe_total': 50.0
    },
    {
        'codigo_pedido': str(uuid.uuid4().int)[:7],
        'fecha': '2023-05-16',
        'codigo_cliente': clientes[1]['ID'],
        'importe_total': 100.0
    },
    {
        'codigo_pedido': str(uuid.uuid4().int)[:7],
        'fecha': '2023-05-16',
        'codigo_cliente': clientes[2]['ID'],
        'importe_total': 200.0
    },
    {
        'codigo_pedido': str(uuid.uuid4().int)[:7],
        'fecha': '2023-05-16',
        'codigo_cliente': clientes[2]['ID'],
        'importe_total': 300.0
    },
]

In [9]:
# Nos conectamos a la base de datos local que creamos.
client = MongoClient('localhost')
print(client.list_database_names())

['Tienda', 'admin', 'config', 'local']


In [22]:
# Creamos nuestra nueva base de datos a)
db = client['Tienda']
# Creamos las colecciones que necesitamos a)
clientes_col = db['clientes']
pedidos_col = db['pedidos']
# Insertamos los clientes uno a uno c)
# He añadido que solo añada el cliente si no está en la base de datos
for cliente in clientes:
    # Comprobamos que el cliente no está en la colección
    if clientes_col.find_one({'ID': cliente['ID']}) is None:
        clientes_col.insert_one(cliente)
# Insertamos los pedidos uno a uno para poder meter el condicional
# y añadirlos solo si no están en la base de datos
for pedido in pedidos:
    # Comprobamos que el pedido no está en la colección
    if pedidos_col.find_one({'codigo_pedido': pedido['codigo_pedido']}) is None:
        pedidos_col.insert_one(pedido)
print('')  # Esto es porque me da toc que haga eco

In [23]:
# Vamos a ver qué hay en nuestra base de datos.
print(client.list_database_names())
print(db.list_collection_names())
print(clientes_col.count_documents({}))
print(pedidos_col.count_documents({}))

['Tienda', 'admin', 'config', 'local']
['clientes', 'pedidos']
3
7


In [25]:
# Leemos todos los clientes y todos los pedidos d)
clients = clientes_col.find()
peds = pedidos_col.find()

# Recorremos todos los clientes
for cliente in clientes:
    # Leemos los pedidos del cliente d)
    id_cliente = cliente['ID']
    pedidos_cliente = pedidos_col.find({'codigo_cliente': id_cliente})

    # Actualizamos los pedidos del cliente
    cliente_pedidos = []
    dinero_gastado = 0.0
    num_pedidos = 0
    for pedido in pedidos_cliente:
        cliente_pedidos.append(pedido)
        dinero_gastado += pedido['importe_total']
        num_pedidos += 1

    # Actualizamos los datos del cliente en la base de datos: e)
    clientes_col.update_one(
        {'ID': id_cliente},
        {'$set': {'pedidos': cliente_pedidos,
                  'dinero_gastado': dinero_gastado,
                  'num_pedidos': num_pedidos}}
    )

In [26]:
# Vamos a imprimirlo de forma legible usando pprint
# Se pueden meter los condicionales de mongo.
for cliente in clientes_col.find({}):
    pprint(cliente)

{'ID': '2131062',
 '_id': ObjectId('64745282e2c7003a057a49f4'),
 'apellidos': 'Cuenca',
 'dinero_gastado': 250.0,
 'nombre': 'Pedro',
 'num_pedidos': 2,
 'pedidos': [{'_id': ObjectId('64745282e2c7003a057a49f7'),
              'codigo_cliente': '2131062',
              'codigo_pedido': '6346583',
              'fecha': '2023-05-18',
              'importe_total': 100.0},
             {'_id': ObjectId('64745282e2c7003a057a49f8'),
              'codigo_cliente': '2131062',
              'codigo_pedido': '1404795',
              'fecha': '2023-05-17',
              'importe_total': 150.0}]}
{'ID': '9400281',
 '_id': ObjectId('64745282e2c7003a057a49f5'),
 'apellidos': 'Martínez',
 'dinero_gastado': 350.0,
 'nombre': 'Ana',
 'num_pedidos': 3,
 'pedidos': [{'_id': ObjectId('64745282e2c7003a057a49f9'),
              'codigo_cliente': '9400281',
              'codigo_pedido': '1359881',
              'fecha': '2023-05-16',
              'importe_total': 200.0},
             {'_id': ObjectId('64

In [19]:
client.drop_database('Tienda')  # para eliminar la base de datos.

In [91]:
client.close()  # Cerramos la conexión con el cliente